In [1]:
# Imports
import pathlib
import os
from dataclasses import dataclass, field
from typing import Dict, Literal, Optional, Tuple, List
import json

import imutils
import numpy as np
import cv2
from dataclasses_json import DataClassJsonMixin
import pandas as pd
from tqdm import tqdm

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent
DATA_DIR = GIT_ROOT / "data" / 'PhotosynthesisFall2022'

In [3]:
# Game state reconstruction routines
@dataclass
class Participant(DataClassJsonMixin):
    id: str
    position: Tuple[float, float]
    state: Literal['null', 'H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O'] = 'null'

@dataclass
class EnvironmentState(DataClassJsonMixin):
    sun_state: Optional[bool] = None

@dataclass
class GameState(DataClassJsonMixin):
    participants: Dict[str, Participant] = field(default_factory=dict)
    environment: EnvironmentState = field(default_factory=EnvironmentState)

@dataclass
class GamePhaseState(DataClassJsonMixin):
    phase: Literal['before_game', 'molecule_select', 'normal', 'paused']

# Additional for state-action sequence generation
@dataclass
class TrackedParticipant:
    timestamp: float
    participant: Participant
    movement_history: List[Tuple[float, float]] = field(default_factory=list)

@dataclass
class StateAction(DataClassJsonMixin):
    timestamp: float
    phase: Literal['molecule_select', 'normal']
    state: Literal['H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O']
    action: Literal['rabbit', 'choro', 'roots', 'plant']
    out_state: Literal['H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O']
    distance: float
    duration: float
    success: bool

CORRECT_TRANSITIONS = {
    "H2O": {
        "choro": "O2",
    },
    "O2": {
        "rabbit": "CO2",
    },
    "CO2": {
        "choro": "Sugar"
    },
    "Sugar": {
        "plant": "Thinking_H2O",
    },
    "Thinking_H2O": {
        "roots": "H2O"
    }
}

# In-game items
ITEMS = {
    "rabbit": {"lt": (50, 370), "rb": (170, 465)},
    "plant": {"lt": (200, 95), "rb": (365, 460)}, # Mostly students don't purposefully transition from Sugar to Thinking_H2O
    "choro1": {"lt": (445, 60), "rb": (545, 170)},
    "choro2": {"lt": (500, 390), "rb": (600, 500)},
    "roots": {"lt": (200, 460), "rb": (365, 560)},
}

In [4]:
# Helper functions
def xy_transforms(xy: Tuple[float, float], w: int, h:int, c: Dict[str, float]) -> Tuple[float, float]:
    x, y = xy
    xx = int(((x+1)*h/2)*c['AFFINE'][0] + c['OFFSET'][0])
    yy = int(((y+1)*w/2)*c['AFFINE'][1] + c['OFFSET'][1])
    return (xx, yy)

def detect_action(xy: Tuple[float, float], w: int, h:int, c: Dict[str, float]) -> Tuple[float, float]:
    xy = xy_transforms(xy, w, h, c)

    # Check if they are inside any of the items
    for item, coords in ITEMS.items():
        if coords['lt'][0] < xy[0] < coords['rb'][0] and coords['lt'][1] < xy[1] < coords['rb'][1]:
            return item
        
    return None

def compute_distance_list(xys: List[Tuple[float, float]]):
    distances = []
    for i in range(len(xys)-1):
        distances.append(np.linalg.norm(np.array(xys[i+1]) - np.array(xys[i])))
    return distances

def render(game_state: GameState, frame: np.ndarray, c: Dict[str, float]):

    if game_state.participants:
        for p in game_state.participants.values():
            p.position = (eval(p.position[0]), eval(p.position[1]))
            xy = xy_transforms(p.position, frame.shape[0], frame.shape[1], c)
            frame = cv2.circle(frame, xy, 30, (0, 0, 255), 1)
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (0,0,0), 
                2, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (255,255,255), 
                1, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (0,0,0), 
                2,
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (255,255,255), 
                1,
                cv2.LINE_AA
            )

    return frame

In [10]:
# Put in ceil to allow debugging
def get_state_action_sequence(timestamp, tracked_participants: Dict[str, TrackedParticipant], game_state: GameState, c: Dict[str, float], game_phase: GamePhaseState) -> Dict[str, StateAction]:
    
    # Check for the game phase
    if game_phase.phase in ['before_game', 'paused']:
        return {}

    # Create container
    state_action_sequence: Dict[str, StateAction] = {}

    for p in game_state.participants.values():
        if p.id not in tracked_participants:
            tracked_participants[p.id] = TrackedParticipant(
                timestamp,
                p
            )
        else:
            # Identify if a transition has occurred
            if tracked_participants[p.id].participant.state != p.state:

                # If occur, compute state-action information
                distance = sum(compute_distance_list(tracked_participants[p.id].movement_history))
                state_action_sequence[p.id] = StateAction(
                    timestamp=timestamp,
                    state=tracked_participants[p.id].participant.state,
                    action=CORRECT_TRANSITIONS[tracked_participants.state],
                    out_state=p.state,
                    distance=distance,
                    duration=timestamp - tracked_participants[p.id].timestamp,
                )

                # Reset information
                tracked_participants[p.id].timestamp = timestamp
                tracked_participants[p.id].movement_history = [p.position]

            else:
                # Identify if an action has occurred (incorrect one)
                action = detect_action(p.position, 1920, 1080, c)
                if action:
                    # If occur, compute state-action information
                    distance = sum(compute_distance_list(tracked_participants[p.id].movement_history))
                    state_action_sequence[p.id] = StateAction(
                        timestamp=timestamp,
                        state=tracked_participants[p.id].participant.state,
                        action=action,
                        out_state=p.state,
                        distance=distance,
                        duration=timestamp - tracked_participants[p.id].timestamp,
                        success=False
                    )

                # Update data
                tracked_participants[p.id].participant.position = p.position
                tracked_participants[p.id].movement_history.append(p.position)

    return state_action_sequence


# Transformation constants
START_INDEX = 6000
CORRECTIONS = {'OFFSET': (-252,-280), 'AFFINE': (1.8,2)}

def state_action_processing(id, vid_file, game_state_file, annotated_events_file):
    
    # Load the data
    assert vid_file.exists()
    cap = cv2.VideoCapture(str(vid_file))
    fps = cap.get(cv2.CAP_PROP_FPS)
    N = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    assert game_state_file.exists()
    game_state_logs = pd.read_csv(game_state_file)

    assert annotated_events_file.exists()
    annotated_events = pd.read_csv(annotated_events_file)

    # Reset video and logs
    cap.set(cv2.CAP_PROP_POS_FRAMES, START_INDEX)
    game_state_pointer = 0
    game_event_pointer = 0

    # Create container for each participant state-action data
    tracked_participants: Dict[str, TrackedParticipant] = {}
    sequences: Dict[str, Dict[str, List[StateAction]]] = {}
    game_phase = GamePhaseState(phase='before_game')

    for i in tqdm(range(N), total=N):
        
        ret, frame = cap.read()
        if not ret:
            break

        frame_timestamp = (START_INDEX + i) / fps
        frame = cv2.putText(
            frame, 
            f"{pd.Timestamp(frame_timestamp, unit='s').strftime('%H:%M:%S.%f')}", 
            (5,30), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (0,0,255), 
                1, 
                cv2.LINE_AA
            )    
        
        # Processing events (like Game Phase)
        while len(annotated_events) > game_event_pointer+1 and annotated_events['timestamp'].iloc[game_event_pointer+1] < frame_timestamp:
            event = annotated_events.iloc[game_event_pointer+1]
            if event.event_type == 'phase':
                data = event.event_data.replace("“", '"')
                data = data.replace("”", '"')
                phase = json.loads(data)['phase']
                assert phase in ['before_game', 'molecule_select', 'normal', 'paused']
                game_phase.phase = phase

            game_event_pointer += 1

        # Obtain game state information
        while game_state_logs['timestamp'].iloc[game_state_pointer+1] < frame_timestamp:
            game_state_pointer += 1
            timestamp = game_state_logs['timestamp'].iloc[game_state_pointer]
            new = GameState.from_json(game_state_logs.iloc[game_state_pointer].state)
            step_sequences = get_state_action_sequence(timestamp, tracked_participants, new, CORRECTIONS, game_phase)
            # step_sequences = {}

            for p, s in step_sequences.items():
                if p not in sequences:
                    sequences[p] = {'state-action': []}
                sequences[p]['state-action'].append(s)

        game_state = GameState.from_json(game_state_logs.iloc[game_state_pointer].state)

        # For macro alignment
        # frame = cv2.putText(
        #     frame, 
        #     f"{game_state_pointer}", 
        #     (5,60), 
        #     cv2.FONT_HERSHEY_SIMPLEX, 
        #     1, 
        #     (0,0,255), 
        #     1, 
        #     cv2.LINE_AA
        # )
        frame = cv2.putText(
            frame, 
            f"{game_phase.phase}", 
            (5,60), 
            cv2.FONT_HERSHEY_SIMPLEX,
            1, 
            (0,0,255), 
            1, 
            cv2.LINE_AA
        )

        # For each item, draw the rectangle
        for item, item_data in ITEMS.items():
            lt = item_data['lt']
            rb = item_data['rb']
            frame = cv2.rectangle(frame, lt, rb, (0, 0, 255), 1)
            frame = cv2.putText(
                frame, 
                item, 
                (lt[0], lt[1]+10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (0,0,255), 
                1, 
                cv2.LINE_AA
            )

        frame = render(game_state, frame, CORRECTIONS)
        cv2.imshow('frame', imutils.resize(frame, width=1000))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

    # Convert and save data
    for p, data in sequences.items():
        df = pd.DataFrame(data)
        df.to_csv(DATA_DIR / 'state_action_sequences' / f'{id}_{p}.csv', index=False)

# Processing - Day 11
state_action_processing(
    'day11', 
    DATA_DIR / 'videos' / 'day 11' / "day11-screen-recording.mp4",
    DATA_DIR / 'time_alignment' / 'day11_aligned_game_state.csv',
    DATA_DIR / 'videos' / 'day 11' / 'annotated_game_events.csv'
)

  2%|▏         | 304/16351 [14:27<12:43:37,  2.86s/it]


KeyboardInterrupt: 

: 